In [ ]:
import random
import json
import numpy as np
from scipy.optimize import linear_sum_assignment
from queue import PriorityQueue

class SimulationEngine:
    def __init__(self, lots, machines, recipe_queues, max_simulation_time):
        self.lots = lots  # Dict {lot_id: Lot instance}
        self.machines = machines  # Dict {machine_id: Machine instance}
        self.recipe_queues = recipe_queues # Dict {recp_id: recp PQ}
        self.general_fel = []  # Global system event log
        self.timestamp = PriorityQueue()  # Global timestamp
        self.currsimtime = 0  # Current simulation time
        self.max_simulation_time = max_simulation_time  # Stop condition
'''
Yilun's note:
this logic is definitely doable, by performaning 1:N assignment in a loop, it gives more choice to those machines in front
that's why it is greedy, and it is solved by hungarian algorithm
thos machine behind has less choice, but it is still possible to get a good solution when the queue is highly imbalanced
'''
def assign_lots_to_machines(idle_machines, machines, recipe_queues,recp_times_func, travel_time_func, switching_cost_func):
    """
    Assign lots to idle machines based on weight calculation and optimization.
    
    Parameters:
    - idle_machines: List of machine IDs that are idle.
    - machines: Dict mapping machine IDs to machine instances.
    - recipe_queues: Dict mapping recipe IDs to priority queues (PriorityQueue of lots ordered by ACR).
    - travel_time_func: Function that takes in (lot.location, machine.location) and returns travel time.
    - switching_cost_func: Function that takes in (machine.currrecpid, recp queue) and returns switching cost.
    """
    assignments = []
    # prepare assignment problem
    machine_list = []
    queue_list = []
    weight_matrix = []
    dummy_penalty = 28800
    
    # Prepare assignment problem
    for machine_id in idle_machines:
        # get instance
        machine = machines[machine_id]
        child = machine.child
        capable_recipes = machine.recipes
        # add idle machine to assignment list
        machine_list.append(machine_id)

        # 1:N, machine to recipe
        row_weights = []
        for recp_id in capable_recipes:
            # queue have at least one lot
            if recp_id in recipe_queues and not recipe_queues[recp_id].empty():
                queue_list.append((machine_id, recp_id))
                # determine number of lots to assign
                lots_to_assign = min(child, recipe_queues[recp_id].qsize())
                # get lots
                selected_lots = [recipe_queues[recp_id].get() for _ in range(lots_to_assign)]
                # get ACR, travel time, switching cost
                acr_vals = [lot.acr for lot in selected_lots]
                recp_times = recp_times_func(recp_id)
                travel_times_vals = [travel_time_func(lot.location, machine.location) for lot in selected_lots]
                max_travel_time = max(travel_times_vals)
                switching_time = switching_cost_func(machine.curr_recipe, recp_id)
                
                # Compute weight
                # (recp + travel + switching)/lots_count*avg(ACR)^2 - dummy_penalty * int(any(ACR < 1))
                weight = (recp_times + switching_time + max_travel_time) / lots_to_assign * ((sum(acr_vals)/len(acr_vals)) ** 2)
                weight = weight - dummy_penalty * int(acr_vals[0]<1)
                row_weights.append(weight)
        # dummy recp for machine to be strategic idling
        queue_list.append((machine_id, "idle"))
        row_weights.append(dummy_penalty)
        weight_matrix.append(row_weights)
    
    weight_matrix = np.array(weight_matrix)
    
    # Solve assignment problem by Hungarian algorithm 
    machine_idx, queue_idx = linear_sum_assignment(weight_matrix)
    
    for m_idx, q_idx in zip(machine_idx, queue_idx):
        machine_id = machine_list[m_idx]
        recp_id, _ = queue_list[q_idx]
        
        if weight_matrix[m_idx, q_idx] < dummy_penalty:
            assignments.append((machine_id, recp_id))
    
    return assignments





### Yilun's note:
this logic is trying M:N assignment, it is more fair to all machines, then solve the global ILP problem
Muhc more complex, and it is computationally expensive
##### still under development


In [ ]:
import random
import json
import numpy as np
import cplex
from queue import PriorityQueue

class SimulationEngine:
    def __init__(self, lots, machines, recipe_queues, max_simulation_time, penalty=28800):
        self.lots = lots  # Dict {lot_id: Lot instance}
        self.machines = machines  # Dict {machine_id: Machine instance}
        self.recipe_queues = recipe_queues # Dict {recp_id: recp PQ}
        self.general_fel = []  # Global system event log
        self.timestamp = PriorityQueue()  # Global timestamp
        self.currsimtime = 0  # Current simulation time
        self.max_simulation_time = max_simulation_time  # Stop condition
        self.penalty = penalty  # Default penalty for idle machines

def assign_lots_to_machines(idle_machines, machines, recipe_queues, recp_times_func, travel_time_func, switching_cost_func, penalty=28800):
    """
    Assign lots to idle machines using M:N global optimization with CPLEX ILP.
    
    Parameters:
    - idle_machines: List of machine IDs that are idle.
    - machines: Dict mapping machine IDs to machine instances.
    - recipe_queues: Dict mapping recipe IDs to priority queues (PriorityQueue of lots ordered by ACR).
    - travel_time_func: Function returning travel time between lot and machine.
    - switching_cost_func: Function returning switching cost.
    - penalty: Cost incurred for machine idling.
    """
    assignments = []
    problem = cplex.Cplex()
    problem.set_problem_type(cplex.Cplex.problem_type.LP)
    
    lots_by_recp = recipe_queues  # Directly use recipe_queues
    
    machine_list = idle_machines
    num_machines = len(machine_list)
    
    # Decision variables
    x = {}  # x[m, recp, l] = 1 if machine m takes lot l from recp queue
    y = {}  # y[m] = 1 if machine is idle
    
    obj_terms = []  # Store objective function terms
    
    for m in machine_list:
        y[m] = problem.variables.add(names=[f"y_{m}"], types=["B"], obj=[penalty])
        for recp_id, lots in lots_by_recp.items():
            if len(lots.queue) > 0:
                acr_value = lots.queue[0].acr  # Use the smallest ACR in queue (first element)
                for l in range(len(lots.queue)):
                    var_name = f"x_{m}_{recp_id}_{l}"
                    x[m, recp_id, l] = problem.variables.add(names=[var_name], types=["B"], obj=[0])
                    recp_time = recp_times_func(recp_id, machines[m])  # Ensure different machines have unique recp times
                    weight = (recp_time + switching_cost_func(machines[m].curr_recipe, recp_id) + 
                              travel_time_func(lots.queue[l].location, machines[m].location)) * (acr_value ** 2)
                    if acr_value < 1:
                        weight -= penalty  # Apply bonus weightage reduction if ACR < 1
                    obj_terms.append((var_name, weight))
    
    # Set the objective function as a sum of all weights * decision variables
    problem.objective.set_linear(obj_terms)
    
    # Constraints: Each lot is assigned at most once
    for recp_id, lots in lots_by_recp.items():
        for l in range(len(lots.queue)):
            problem.linear_constraints.add(
                lin_expr=[[x[m, recp_id, l] for m in machine_list], [1] * num_machines],
                senses=['L'], rhs=[1])
    
    # Constraints: Each machine takes from at most one recipe queue
    for m in machine_list:
        problem.linear_constraints.add(
            lin_expr=[[x[m, recp_id, l] for recp_id in lots_by_recp for l in range(len(lots_by_recp[recp_id].queue))], 
                      [1] * sum(len(lots.queue) for lots in lots_by_recp.values())],
            senses=['L'], rhs=[machines[m].child])
    
    # Constraints: Total lots taken from a queue do not exceed queue length
    for recp_id, lots in lots_by_recp.items():
        problem.linear_constraints.add(
            lin_expr=[[x[m, recp_id, l] for m in machine_list for l in range(len(lots.queue))], 
                      [1] * (num_machines * len(lots.queue))],
            senses=['L'], rhs=[len(lots.queue)])
    
    # Constraints: A machine is idle if no lots are assigned
    for m in machine_list:
        problem.linear_constraints.add(
            lin_expr=[[x[m, recp_id, l] for recp_id in lots_by_recp for l in range(len(lots_by_recp[recp_id].queue))] + [y[m]],
                      [1] * sum(len(lots.queue) for lots in lots_by_recp.values()) + [1]],
            senses=['E'], rhs=[1])
    
    problem.solve()
    
    # Extract results
    solution_values = problem.solution.get_values()
    for m in machine_list:
        for recp_id, lots in lots_by_recp.items():
            for l in range(len(lots.queue)):
                if solution_values[f"x_{m}_{recp_id}_{l}"] > 0.5:
                    assignments.append((m, recp_id, list(lots.queue)[l].id))
    
    return assignments
